<h1>New Experiment 2: Random rejection of motion-free subjects</h1>
In this notebook, random rejection tests were conducted using data from S1 in the still condition. The generalisation tests were conducted over S2. Each time we fix the number of retained volumes N. We randomly select N volumes from a subject's data, then we train our proposed model over this sample. At testing, we applied the same rejection scheme to S2, then test over the subset of S2.</br>
We would like to show this proposed model is robust, by which it is independent of 1. The proportion of discarded volumes and 2. The different combinations of b-values.</br>
<h2>Packages</h2>

In [ ]:
import nibabel as nib
import numpy as np
import os
"""
packages that does conventional model fitting
"""
import amico
"""
packages that generate train/test dataset
"""
from FormatData import generate_data, parser as data_parser
"""
packages that trains network
"""
from Training import train_network
from utils.model import parser as model_parser
"""
packages that test network
"""
from Testing import test_model
"""
packages that produce the rejection shceme
"""
from filter_qa import parser as filter_parser, load_eddy
"""
package to store the intermediate result
"""
import pickle

In [ ]:
"""
packages that handle graphs
"""
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm, LinearSegmentedColormap
from skimage.metrics import structural_similarity as compare_ssim
from utils import calc_ssim
%matplotlib inline
def plot_loss(cmd):
    """
    A function that used to plot the loss curve for the trained network.
    Args:
        cmd: String, the command line in the terminal
    """
    args = model_parser().parse_args(cmd.split())
    history = train_network(args)
    # plt.plot(history.history['loss'])
    # plt.plot(history.history['val_loss'])
    # plt.title('model loss')
    # plt.ylabel('loss')
    # plt.xlabel('epoch')
    # plt.legend(['train', 'validation'], loc='upper left')
    # plt.show()

def show_slices(slices, grayscale=True):
    """
    Function to display the slices

    Args:
        slices (list): a list of 2d ndarray that contains the data to be displayed
        grayscale (bool, optional): True, if diplay grayscale img. Defaults to True.
    """    
    fig, axes = plt.subplots(1, len(slices), figsize=(10,10))
    cax = fig.add_axes([0, 0, .3, .3])
    for i, slice in enumerate(slices):
        # use grayscale for displaying ref and pred imgs:
        if grayscale:
            cmap = plt.get_cmap('gray')
            cmaplist = [cmap(i) for i in range(cmap.N)]
            cmap = LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            bounds = np.arange(0, 1.0, .01)
            idx = np.searchsorted(bounds, 0)
            bounds = np.insert(bounds, idx, 0)
            norm = BoundaryNorm(bounds, cmap.N)
            im = axes[i].imshow(slice.T, cmap=cmap, origin="lower", interpolation='none', norm=norm)
        else:
            # define the colormap
            cmap = plt.get_cmap('bwr')
            # extract all colors from the .jet map
            cmaplist = [cmap(i) for i in range(cmap.N)]
            # create the new map
            cmap = LinearSegmentedColormap.from_list('Custom cmap', cmaplist, cmap.N)
            # define the bins and normalize and forcing 0 to be part of the colorbar
            # define the min and max to be -1 and +1 respectively
            bounds = np.arange(-0.5, 0.5, .01)
            idx = np.searchsorted(bounds, 0)
            bounds = np.insert(bounds, idx, 0)
            norm = BoundaryNorm(bounds, cmap.N)
            im = axes[i].imshow(slice.T, cmap=cmap, origin="lower", interpolation='none', norm=norm)
        fig.colorbar(im, cax=cax, orientation='vertical')

def scale(img):
    # for i in range(img.shape[0]):
    #     for j in range(img.shape[1]):
    #         img[i][j] = ((img[i][j]+1)/2)*255
    return img

def compare_simi(pred, ref):
    return calc_ssim(pred, ref)

<h2>The class for parsing command line arguments</h2>

In [ ]:
class Namespace:
    """
    a class generate parser for cmd line args
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

------------------------
<h2>Random Rejection Scheme Setup</h2>
Let N to be the tested number of retained volumes. For N, 100 subsampled schemes were drawn randomly from the full scheme (with the first b=0 volume and at least two different b values always included). Each subsampled schem was used to evaluate both techniques. For both 3D CNN and AMICO, N are 60, 40 and 30. We would give further undersampled scheme to 3D CNN, N are 20, 16 and 12.

In [ ]:
# bvals = '0 3000 1000 2000 2000 1000 2000 3000 1000 2000 3000 1000 2000 3000 1000 2000 3000 0 1000 3000 2000 1000 2000 3000 1000 2000 3000 1000 2000 3000 1000 2000 3000 0 1000 3000 2000 1000 1000 2000 3000 2000 3000 1000 2000 3000 1000 2000 3000 0 1000 2000 3000 1000 2000 3000 1000 2000 3000 1000 2000 3000 2000 1000 3000 0 1000 3000 2000 3000 1000 2000 1000 2000 3000 3000 1000 2000 1000 2000 3000 0 1000 3000 2000 1000 2000 2000 1000 3000 3000 1000 2000 3000 3000 1000'
# bvals = np.asarray(bvals.split())
# b1000 = np.where(bvals=='1000')
# b2000 = np.where(bvals=='2000')
# b3000 = np.where(bvals=='3000')
# b0 = np.where(bvals=='0')

In [ ]:
# a list contains the N for each tested retained volume
# N = [60, 40, 30, 20, 16, 12]
# random_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}
# random_dict

In [ ]:
# Generated the random scheme for each tested volume
# for n in N:
#     # for each retained volume, we create 100 subsamples
#     for i in range(100):
#         ones = np.ones(n-1) # 1 = retained
#         zeros = np.zeros(96-n)

#         scheme = np.random.choice(np.concatenate([ones,zeros]), 95, replace=False)
#         # always select the first b=0 volume
#         scheme = np.insert(scheme, 0, 1.0)
#         select_index = np.where(scheme==1)
#         # check if 1000 is included
#         has1000 = np.intersect1d(select_index, b1000)
#         has2000 = np.intersect1d(select_index, b2000)
#         has3000 = np.intersect1d(select_index, b3000)
#         if len(has1000) == 0:
#             scheme[b1000[0]] = 1
#             if len(has2000) > 1:
#                 # remove a 2000 b val volume
#                 scheme[has2000[0]] = 0
#             else:
#                 scheme[has3000[0]] = 0
#         if len(has2000) == 0:
#             scheme[b2000[0]] = 1
#             if len(has1000) > 1:
#                 # remove a 2000 b val volume
#                 scheme[has1000[0]] = 0
#             else:
#                 scheme[has3000[0]] = 0
#         if len(has3000) == 0:
#             scheme[b3000[0]] = 1
#             if len(has1000) > 1:
#                 # remove a 2000 b val volume
#                 scheme[has1000[0]] = 0
#             else:
#                 scheme[has2000[0]] = 0
#         scheme = ' '.join(map(str, scheme))
#         random_dict[n].append(scheme)

In [ ]:
# # save the rejection scheme
# random_60_file = 'random_60.pickle'
# random_40_file = 'random_40.pickle'
# random_30_file = 'random_30.pickle'
# random_20_file = 'random_20.pickle'
# random_16_file = 'random_16.pickle'
# random_12_file = 'random_12.pickle'

# with open(random_60_file, 'wb') as handle:
#     pickle.dump(random_dict[60], handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(random_40_file, 'wb') as handle:
#     pickle.dump(random_dict[40], handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(random_30_file, 'wb') as handle:
#     pickle.dump(random_dict[30], handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(random_20_file, 'wb') as handle:
#     pickle.dump(random_dict[20], handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(random_16_file, 'wb') as handle:
#     pickle.dump(random_dict[16], handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open(random_12_file, 'wb') as handle:
#     pickle.dump(random_dict[12], handle, protocol=pickle.HIGHEST_PROTOCOL)

If the random files have already been created, just read those files

In [ ]:
random_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}

In [ ]:
random_fp = '/home/vw/Data/IndividualProject/MedICSS2021_/Net/new_exp2_results'

with open(os.path.join(random_fp, 'random_60.pickle'), 'rb') as file:
    while True:
        try:
            random_dict[60].append(pickle.load(file))
        except EOFError:
            break

with open(os.path.join(random_fp, 'random_40.pickle'), 'rb') as file:
    while True:
        try:
            random_dict[40].append(pickle.load(file))
        except EOFError:
            break

with open(os.path.join(random_fp, 'random_30.pickle'), 'rb') as file:
    while True:
        try:
            random_dict[30].append(pickle.load(file))
        except EOFError:
            break

with open(os.path.join(random_fp, 'random_20.pickle'), 'rb') as file:
    while True:
        try:
            random_dict[20].append(pickle.load(file))
        except EOFError:
            break

with open(os.path.join(random_fp, 'random_16.pickle'), 'rb') as file:
    while True:
        try:
            random_dict[16].append(pickle.load(file))
        except EOFError:
            break
with open(os.path.join(random_fp, 'random_12.pickle'), 'rb') as file:
    while True:
        try:
            random_dict[12].append(pickle.load(file))
        except EOFError:
            break

random_dict[60] = np.array(random_dict[60][0])
random_dict[40] = np.array(random_dict[40][0])
random_dict[30] = np.array(random_dict[30][0])
random_dict[20] = np.array(random_dict[20][0])
random_dict[16] = np.array(random_dict[16][0])
random_dict[12] = np.array(random_dict[12][0])

In [ ]:
cnn_rmse_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}
cnn_ssim_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}
cnn_percent_dict = {60:[], 40:[], 30:[], 20:[], 16:[], 12:[]}

----------------------------
<h2>Training</h2>
Hyperparameters

In [ ]:
# hyperparameters for the networks
layers = 4 # the number of hidden layers; this is the optimal number of layer
lr = 0.0001 # the learning rate
patch_size = 3 # the size of patches for 2D and 3D CNN
batch = 256 # the batch size
epoch = 100 # the number of epoches for training
model = 'conv3d'
ltype = 'A'

# we firstly study subject s02, use s01 as a seperate training dataset
study_subject = 's03_still_reg' # the subject we want to study
sep_train_subject = 's01_still' # this is separate training candidate

<h4>N = 60</h4>

In [ ]:
for i in range(len(random_dict[60])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[60][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[60].append(RMSE)
    cnn_ssim_dict[60].append(SSIM)
    cnn_percent_dict[60].append(Percent)

In [ ]:
with open('cnn_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[60][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_dict[60])//2, len(random_dict[60])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[60][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[60].append(RMSE)
    cnn_ssim_dict[60].append(SSIM)
    cnn_percent_dict[60].append(Percent)

In [ ]:
with open('cnn_rmse_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_60_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_60_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_60_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_60_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_60_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[60][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 40</h4>

In [ ]:
for i in range(len(random_dict[40])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[40][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[40].append(RMSE)
    cnn_ssim_dict[40].append(SSIM)
    cnn_percent_dict[40].append(Percent)

In [ ]:
with open('cnn_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[40][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_dict[40])//2, len(random_dict[40])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[40][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[40].append(RMSE)
    cnn_ssim_dict[40].append(SSIM)
    cnn_percent_dict[40].append(Percent)

In [ ]:
with open('cnn_rmse_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_40_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_40_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_40_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_40_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_40_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[40][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 30</h4>

In [ ]:
for i in range(len(random_dict[30])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[30][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[30].append(RMSE)
    cnn_ssim_dict[30].append(SSIM)
    cnn_percent_dict[30].append(Percent)

In [ ]:
with open('cnn_rmse_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[30][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_dict[30])//2, len(random_dict[30])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[30][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[30].append(RMSE)
    cnn_ssim_dict[30].append(SSIM)
    cnn_percent_dict[30].append(Percent)

In [ ]:
with open('cnn_rmse_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_30_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_30_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_30_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_30_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_30_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[30][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 20</h4>

In [ ]:
for i in range(len(random_dict[20])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[20][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[20].append(RMSE)
    cnn_ssim_dict[20].append(SSIM)
    cnn_percent_dict[20].append(Percent)

In [ ]:
with open('cnn_rmse_20_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_20_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_20_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_20_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_20_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[20][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_dict[20])//2, len(random_dict[20])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[20][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[20].append(RMSE)
    cnn_ssim_dict[20].append(SSIM)
    cnn_percent_dict[20].append(Percent)

In [ ]:
with open('cnn_rmse_20_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_20_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_20_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_20_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_20_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_20_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_20_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[20][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 16</h4>

In [ ]:
for i in range(len(random_dict[16])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[16][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[16].append(RMSE)
    cnn_ssim_dict[16].append(SSIM)
    cnn_percent_dict[16].append(Percent)

In [ ]:
with open('cnn_rmse_16_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_16_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_16_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_16_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_16_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[16][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
for i in range(len(random_dict[16])//2, len(random_dict[16])):
    # we have 100 movefiles for each retained tested volume
    # Training
    movefile = random_dict[16][i]
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[16].append(RMSE)
    cnn_ssim_dict[16].append(SSIM)
    cnn_percent_dict[16].append(Percent)

In [ ]:
with open('cnn_rmse_16_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_16_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_16_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_16_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_16_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_16_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_16_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[16][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

<h4>N = 12</h4>

In [ ]:
# for i in range(len(random_dict[12])//2):
for i in range(len(random_dict[12])//2):
    # we have 100 movefiles for each retained tested volume
    # Training
    # movefile = random_dict[12][i]
    movefile = random_dict[12][i]
    # movefile = movefile.split()
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[12].append(RMSE)
    cnn_ssim_dict[12].append(SSIM)
    cnn_percent_dict[12].append(Percent)

In [ ]:
with open('cnn_rmse_12_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_12_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_12_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_12_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_12_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[12][:50]], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# for i in range(len(random_dict[12])//2, len(random_dict[12])):
for i in range(len(random_dict[12])//2, len(random_dict[12])):
    # we have 100 movefiles for each retained tested volume
    # Training
    # movefile = random_dict[12][i]
    movefile = random_dict[12][i]
    # movefile = movefile.split()
    cnntrain_cmd = '--train_subjects ' + sep_train_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --lr ' + str(lr) + ' --batch ' + str(batch) + ' --patch_size ' + str(patch_size)\
               + ' --epoch ' + str(epoch) + ' --movefile ' + movefile + ' --train'
    plot_loss(cnntrain_cmd)
    # Testing
    cnntest_cmd = '--test_subjects ' + study_subject + ' --model ' + model + ' --layer ' + str(layers) + ' --label_type ' + ltype + ' --movefile ' + movefile
    cnntest_args = model_parser().parse_args(cnntest_cmd.split())
    RMSE, SSIM, Percent = test_model(cnntest_args)
    # Add results to the cnn dict
    cnn_rmse_dict[12].append(RMSE)
    cnn_ssim_dict[12].append(SSIM)
    cnn_percent_dict[12].append(Percent)

In [ ]:
with open('cnn_rmse_12_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_rmse_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_rmse_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_rmse_12_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_rmse_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_ssim_12_ndi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_ssim_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_odi_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_ssim_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_ssim_12_fwf_'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_ssim_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)

with open('cnn_percent_12_ndi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[0] for item in cnn_percent_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_12_odi'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[1] for item in cnn_percent_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)
with open('cnn_percent_12_fwf'+str(i)+'.pickle', 'wb') as file:
    pickle.dump([item[2] for item in cnn_percent_dict[12][50:]], file, protocol=pickle.HIGHEST_PROTOCOL)